In [1]:
import pandas as pd 
import numpy as np 
import pickle 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Bank_Customer_Churn_Prediction.csv')
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop('customer_id', axis= 1, inplace= True)
data.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Label Encoder of Gender 

label_encoder_gen = LabelEncoder()
data['gender'] = label_encoder_gen.fit_transform(data['gender'])

In [6]:
# One hot encoder of country 

onehot_encoder_con = OneHotEncoder(handle_unknown= 'ignore')
geo_encoded = onehot_encoder_con.fit_transform(data[['country']]).toarray() 
geo_encoded_df = pd.DataFrame(geo_encoded, columns= onehot_encoder_con.get_feature_names_out(['country']))
geo_encoded_df

,country_France,country_Germany,country_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
# Combine one-hat with original 

data = pd.concat([data.drop('country', axis= 1), geo_encoded_df], axis= 1)
data.head()

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
# Split the data into X, y 

X = data.drop('estimated_salary', axis = 1)
y = data['estimated_salary'] 

In [14]:
## Split the data into train and test sets 

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.25, random_state= 42)

## Scale the features 
scaler_reg = StandardScaler()
X_train = scaler_reg.fit_transform(X_train)
X_test = scaler_reg.transform(X_test) 

In [16]:
# save all the pickle files 

with open('label_encoder_gen_reg.pkl', 'wb') as file: 
    pickle.dump(label_encoder_gen, file)


with open('onehot_encoder_con_reg.pkl', 'wb') as file: 
    pickle.dump(onehot_encoder_con, file)

with open('scaler_reg.pkl', 'wb') as file: 
    pickle.dump(scaler_reg, file)

## ANN Implementation for Regression

In [17]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [18]:
# Build the model 

model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1], )), ## HL 1 connected with i/p 
    Dense(32, activation = 'relu'), ## HL 2
    Dense(1) ## O/p layer 
])

## compile the model 
model.compile(optimizer = "adam", loss= "mean_absolute_error", metrics = ['mae'])

model.summary()

2025-04-16 16:37:33.964734: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-16 16:37:33.965103: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-16 16:37:33.965125: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-16 16:37:33.965419: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-16 16:37:33.965851: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
import datetime
# Setup tensorboard 
log_dir= "reg_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tensorflow_callback = TensorBoard(log_dir= log_dir, histogram_freq= 1)

In [21]:
## Setup Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience= 10, restore_best_weights= True)

In [22]:
# Train the model 
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs = 100, batch_size = 32,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


2025-04-16 16:39:50.037715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


235/235 [==============================] - 5s 12ms/step - loss: 100661.2969 - mae: 100661.2969 - val_loss: 98331.7188 - val_mae: 98331.7188
Epoch 2/100
235/235 [==============================] - 1s 6ms/step - loss: 100591.3594 - mae: 100591.3594 - val_loss: 98211.8359 - val_mae: 98211.8359
Epoch 3/100
235/235 [==============================] - 2s 7ms/step - loss: 100423.9375 - mae: 100423.9375 - val_loss: 97998.9219 - val_mae: 97998.9219
Epoch 4/100
235/235 [==============================] - 1s 6ms/step - loss: 100169.1641 - mae: 100169.1641 - val_loss: 97702.0312 - val_mae: 97702.0312
Epoch 5/100
235/235 [==============================] - 2s 7ms/step - loss: 99825.3828 - mae: 99825.3828 - val_loss: 97304.7578 - val_mae: 97304.7578
Epoch 6/100
235/235 [==============================] - 2s 8ms/step - loss: 99380.3984 - mae: 99380.3984 - val_loss: 96809.7031 - val_mae: 96809.7031
Epoch 7/100
235/235 [==============================] - 2s 9ms/step - loss: 98842.3281 - mae: 98842.3281 - val

In [23]:
## Load TensorBoard extension 

%load_ext tensorboard 

In [25]:
%tensorboard --logdir reg_logs/fit

Reusing TensorBoard on port 6006 (pid 14014), started 0:00:09 ago. (Use '!kill 14014' to kill it.)

In [26]:
!kill 14014

In [27]:
# Evaluate the model on the test data 

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

79/79 [==============================] - 0s 4ms/step - loss: 49678.4258 - mae: 49678.4258
Test MAE: 49678.42578125


In [28]:
model.save('regression_model.h5')

/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
